In [ ]:
import numpy as np

In [ ]:
def _grad_mag(im: np.ndarray) -> np.ndarray:
    gx = np.zeros_like(im, dtype=np.float64); gy = np.zeros_like(im, dtype=np.float64)
    gx[:, 1:-1] = 0.5*(im[:, 2:] - im[:, :-2])
    gy[1:-1, :] = 0.5*(im[2:, :] - im[:-2, :])
    return np.sqrt(gx*gx + gy*gy)

def _mad_std(x: np.ndarray) -> float:
    med = np.median(x)
    return 1.4826 * np.median(np.abs(x - med))

def calibrate_moge_noise_params(
    pairs,  # list of dicts: {'rho_pred','rho_gt','verts_gt','normals_gt','cam_centre','look_dir'}
    depth_bins: int = 8,
    edge_high_q: float = 0.90,
    edge_low_q: float = 0.20,
    grazing_front_thresh: float = 0.95,
):
    r_all = []; dtil_all = []; g_all = []; costh_all = []; valid_counts = 0
    lf_stds = []

    for P in pairs:
        rho_pred = P['rho_pred']; rho_gt = P['rho_gt']
        Vgt = P['verts_gt']; Ngt = P['normals_gt']
        C = np.asarray(P['cam_centre']).reshape(1,3).astype(np.float64)
        L = np.asarray(P['look_dir']).reshape(3).astype(np.float64); L /= (np.linalg.norm(L)+1e-12)

        assert rho_pred.shape == rho_gt.shape == Ngt.shape[:2] == Vgt.shape[:2]
        H, W = rho_gt.shape

        valid = np.isfinite(rho_pred) & np.isfinite(rho_gt) & (rho_gt > 0)
        if not np.any(valid): continue
        valid_counts += int(valid.sum())

        # log residuals
        z_pred = np.log(np.maximum(rho_pred, 1e-12))
        z_gt   = np.log(np.maximum(rho_gt,   1e-12))
        r = (z_pred - z_gt)[valid]

        # normalised depth
        rho = rho_gt[valid]
        rho_med = np.median(rho)
        dtil = rho / max(rho_med, 1e-12)

        # depth gradient
        g = _grad_mag(rho_gt)[valid]

        # grazing cos(theta) from GT normals and viewing rays
        R = (Vgt - C).reshape(-1,3)
        ray = R / (np.linalg.norm(R, axis=1, keepdims=True) + 1e-12)
        ray = ray.reshape(H, W, 3)
        cos_th = np.abs(np.sum(Ngt * (-ray), axis=2))[valid]

        r_all.append(r); dtil_all.append(dtil); g_all.append(g); costh_all.append(cos_th)

        # low-frequency std via blurred residuals (ignore NaNs crudely)
        rmap = np.full_like(rho_gt, np.nan, dtype=np.float64); rmap[valid] = r
        m = np.isfinite(rmap); tmp = np.where(m, rmap, 0.0)
        k = 9; pad = k//2
        csum = np.pad(tmp, pad, mode='edge').cumsum(0).cumsum(1)
        csum_m = np.pad(m.astype(np.float64), pad, mode='edge').cumsum(0).cumsum(1)
        num = (csum[k:,k:] - csum[:-k,k:] - csum[k:,:-k] + csum[:-k,:-k])
        den = (csum_m[k:,k:] - csum_m[:-k,k:] - csum_m[k:,:-k] + csum_m[:-k,:-k])
        blur = np.where(den>0, num/den, 0.0)
        lf_stds.append(np.nanstd(blur[m]))

    if valid_counts == 0:
        raise ValueError("No valid pixels across provided pairs.")

    r_all = np.concatenate(r_all); dtil_all = np.concatenate(dtil_all)
    g_all = np.concatenate(g_all); costh_all = np.concatenate(costh_all)

    # Robust scaling for edge normalisation
    g90 = np.quantile(g_all, edge_high_q)
    g_norm = np.clip(g_all / (g90 + 1e-12), 0.0, 1.0)

    # Fit a0,a1 on easy pixels (low-edge & near-front)
    easy = (g_norm < edge_low_q) & (costh_all > grazing_front_thresh)
    if not np.any(easy):  # fallback: lowest 30% edges
        easy = g_norm < 0.30
    q = np.linspace(0.0, 1.0, depth_bins+1)
    qs = np.quantile(dtil_all[easy], q)
    d_centres = []; mad_vals = []
    for i in range(depth_bins):
        m = easy & (dtil_all >= qs[i]) & (dtil_all < qs[i+1])
        if not np.any(m): continue
        d_centres.append(np.median(dtil_all[m]))
        mad_vals.append(_mad_std(r_all[m]))
    A = np.c_[np.ones(len(d_centres)), np.array(d_centres)]
    sol, *_ = np.linalg.lstsq(A, np.array(mad_vals), rcond=None)
    a0 = float(max(sol[0], 1e-6)); a1 = float(max(sol[1], 0.0))

    # Edge multiplier via MAD ratio (high-edge vs low-edge)
    low_edge = g_norm < edge_low_q
    high_edge = g_norm >= 0.9
    m_low = _mad_std(r_all[low_edge]) if np.any(low_edge) else np.nan
    m_high= _mad_std(r_all[high_edge]) if np.any(high_edge) else np.nan
    if np.isfinite(m_low) and np.isfinite(m_high) and m_low > 1e-12:
        edge_mult = float(np.clip(m_high / m_low, 1.5, 5.0))
    else:
        edge_mult = 3.0

    # Grazing lambda from low-edge bins vs (1 - cos(theta))
    mask_g = low_edge
    x = (1.0 - np.clip(costh_all[mask_g], 0.0, 1.0))
    # predicted base sigma without edge/grazing
    sigma_base = a0 + a1 * dtil_all[mask_g]
    sigma_base = np.maximum(sigma_base, 1e-6)
    # Bin by x and compute MAD ratios
    xb = np.quantile(x, [0.0, 0.25, 0.5, 0.75, 1.0])
    Xc, Y = [], []
    for i in range(len(xb)-1):
        sel = (x >= xb[i]) & (x < xb[i+1])
        if not np.any(sel): continue
        mad_r = _mad_std(r_all[mask_g][sel])
        med_sb= np.median(sigma_base[sel])
        if med_sb <= 0: continue
        y = mad_r / med_sb
        Xc.append(np.median(x[sel])); Y.append(y)
    # Fit y =~ 1 + lambda x, clamp
    if len(Xc) >= 2:
        A = np.c_[np.ones(len(Xc)), np.array(Xc)]
        b = np.array(Y)
        sol, *_ = np.linalg.lstsq(A, b, rcond=None)
        grazing_lambda = float(np.clip(sol[1], 0.0, 5.0))
    else:
        grazing_lambda = 1.0

    # Low-frequency std
    corr_std_log = float(np.median(lf_stds)) if lf_stds else 0.015

    # Global inflation corr_inflate for ~68% coverage (one-step scaling)
    # Predicted sigma without corr-field: (a0+a1*dtil)*f_edge*f_graz
    f_edge = 1.0 + (edge_mult - 1.0) * g_norm
    f_graz = 1.0 + grazing_lambda * (1.0 - np.clip(costh_all, 0.0, 1.0))
    f_graz = np.minimum(f_graz, 3.0)
    sigma_pred = (a0 + a1 * dtil_all) * f_edge * f_graz
    sigma_pred = np.maximum(sigma_pred, 1e-6)
    cover = float(np.mean(np.abs(r_all) <= sigma_pred))
    corr_inflate = float(np.clip((0.68 / max(cover, 1e-3)), 1.0, 2.5))

    return {
        "a0": a0, "a1": a1,
        "edge_mult": edge_mult,
        "grazing_lambda": grazing_lambda,
        "corr_std_log": corr_std_log,
        "corr_inflate": corr_inflate,
    }
